In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import requests
import snowflake

import sqlalchemy
from sqlalchemy import create_engine

import datetime
from datetime import date
from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# use your own credentials
username='username'
useremail='email'
password='SBDpassword'

## Below is the code to link SnowFlake Database, do not change

In [ ]:
def get_oauth_token(svc_username=None, svc_password=None):

    if svc_username and svc_password:
        username = svc_username
        password = svc_password
    else:
        print("Username and password isn't provided. Requesting user...")
        username = os.environ["service_account_id"]
        password = get_password_util(
                pass_value=os.environ.get("service_account_password", None),
                pass_name="Please Enter Service Account Password:  ",
            )


    r = requests.post(
        "https://ssoprod.sbdinc.com/as/token.oauth2",
        data={
            "client_id": "Snowflake",
            "grant_type": "password",
            "username": username,
            "password": password,
            "client_secret": 'f9sq630wmLP6UjpSsOk7kTuP6xccCrSOC4YhE1VdTq3GCupqR7gjYcpuhEGRJ9e0',
            "scope": "session:role-any",
        },
    )
    r.raise_for_status()
    access_token = r.json()["access_token"]
    return access_token


In [ ]:
def getsnowflakecursor(snowflakedatabase,
                       snowflakewarehouse,
                       snowflakeschema,
                       snowflakerole,
                       username,
                       userpassword
                      ):
    import requests
    import snowflake.connector
    
    

    access_token = get_oauth_token(svc_username=username,
                               svc_password=userpassword)

    # Gets the version
    activesnowflakeconnector = snowflake.connector.connect(account='sbd_caspian.us-east-1', 
                                                           authenticator='oauth', 
                                                           token=access_token,
                                                          warehouse=snowflakewarehouse,
                                                          database=snowflakedatabase,
                                                          role=snowflakerole,
                                                          schema=snowflakeschema)
    activesnowflakecursor = activesnowflakeconnector.cursor()
    


    sqlalchemyengine = create_engine(f"snowflake://sbd_caspian.us-east-1.snowflakecomputing.com", creator=lambda: activesnowflakeconnector)
    

    return(activesnowflakeconnector,activesnowflakecursor,sqlalchemyengine)


In [ ]:
def get_spark_configs(snowflakedatabase,
                       snowflakewarehouse,
                       snowflakeschema,
                       snowflakerole,
                       username,
                       userpassword,
                     useremail):
    
    
    spark_snowflake_configs = dict(
                sfUrl = 'sbd_caspian.us-east-1.snowflakecomputing.com',
                sfUser = useremail,  
                sfAuthenticator = "oauth",
                sfRole = snowflakerole,
                sfDatabase = snowflakedatabase,
                sfSchema = snowflakeschema,
                sfWarehouse = snowflakewarehouse,  
                sfToken=get_oauth_token(username,userpassword)
                )
    return(spark_snowflake_configs)

In [ ]:
get_oauth_token(username, password)

## You can modify the cell below (dabase, role, schemas) before reading tables

In [ ]:
activesnowflakeconnector,activesnowflakecursor,sqlalchemyengine = getsnowflakecursor(snowflakedatabase='PROD_EDW',
                                                                                     snowflakewarehouse='DEV_AIDA_WH',
                                                                                     snowflakeschema = 'DIMENSIONS',
                                                                                     snowflakerole = 'OPERATIONS_CREATOR_RO',
                                                                                     username = username,
                                                                                     userpassword = password
                                                                                    )

In [ ]:
# DIM_Product table is a huge table even it is smaller anc simplified vs. the consolidated product table, 
# select the column has the useful information in the SQL equiry
# only select based on the majoy SAP, if you need update, change the SQL equiry conditions
queryp = """
select PROD_KEY
, SRC_SYS_KEY
, EFF_DTE
, PROD_ID
, PRIM_PLANT_LOC_CD
, PRIM_PLANT_LOC_DESC
, PROD_EAN_NBR
, PROD_NAME
, PROD_DESC
, PROD_TYP_LKEY
, BRAND_LKEY
, SUPER_SBU_GPP
, SUPER_SBU_GPP_DESC
, GPP_SBU_ID
, GPP_SBU_DESC
, GPP_CATEGORY_DESC
, GPP_PORTFOLIO_ID
, GPP_PORTFOLIO_DESC
, PROD_HIER_LVL_1_DESC
, PROD_HIER_LVL_2_DESC
, PROD_HIER_LVL_3_DESC
, PROD_HIER_LVL_4_DESC
, PROD_HIER_LVL_5_DESC
, PROD_HIER_LVL_6_DESC
, LEN
, VOL
, WDTH
, WGT
, NET_WGT
, HGT
, BASE_UOM_KEY
, BASE_UNIT_OF_MEAS_NAME
, SZ_UOM_KEY
, VOL_UOM_KEY
, WGT_UOM_KEY
, PUR_UOM_KEY
, BOX_QTY
, CTN_QTY
, MSTR_PCK_QTY
, CASE_QTY
, PALLET_QTY
, BAG_QTY
, PACK_QTY
, PROD_DEL_DTE
, LAUNCH_DTE
, PROD_CREATE_DTE
, MATL_GRP_LKEY
, MFGR_NBR
, MFGR_PROD_NBR
from prod_edw.dimensions.dim_product
where src_sys_key like 'SAPE03' or src_sys_key like 'SAPC11' or 
src_sys_key like 'SAPP10' or src_sys_key like 'SAPSHP' or src_sys_key like 'QADCH' or 
src_sys_key like 'QADAR' or src_sys_key like 'QADBR' or src_sys_key like 'QADPE' 
"""

In [ ]:
df1 = pd.read_sql(queryp,sqlalchemyengine)
df1.head()

In [ ]:
df1.shape

#### The product table is a huge table, it is not effcient to export to a CSV file, the better way is to create a list of product SKU
#### wanted to check, and update the following cells to only look at the products in scope, keep in mind same product can have info
#### from different SAP system, and this may include outdated info as well 

In [ ]:
df1[df1['prod_id'] == 'DCST922P1']

In [ ]:
df1[df1['prod_typ_lkey']=='FERT']['prod_id'].nunique()